# IPA-based syllabification of Polish texts

## I. Preliminary steps

### Import necessary libraries and set up the working directory

In [1]:
### Necessary libraries
import os 
import pandas as pd
import IPython # To be able to run magic commands within Jupyter
from pyndl import preprocess
import regex
import gzip

### Set working directory
TOP = '/media/Syllabifiers_repo/'
WD = '/media/Syllabifiers_repo/package/'
os.chdir(WD)

### Import the local 'syllabifier_ipa' package 
import syllabifiers.ipa as sy

# Display option for dataframes
pd.set_option('display.max_colwidth', 100) # Max width of columns when dispalying datasets
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20

### Define file paths

In [2]:
# List of Enlish words and sentences
POLISH_WORDS = TOP + "illustrative_examples/Data_pol/Polish_words.csv"
POLISH_SENTS = TOP + "illustrative_examples/Data_pol/Polish_sentences.csv"
# Main corpus: a sample of 1000 sentences from the Araneum corpus
CORPUS_POL =  TOP + 'illustrative_examples/Data_pol/Araneum_NoTags_1Ksample.txt' 
# locations of the event files that will be created after the different simulations
S2L_EVENTS_POL = TOP + 'illustrative_examples/Results_pol/S2L_event_pol.gz'

### Define some useful variables for later

In [3]:
### Set of allowed Polish characters
POLISH = "aąbcćdeęfghijklłmnńoóprsśtuwyzźżqvx"
POLISH = POLISH + POLISH.upper()

### Compiled regular expression that detects syllabic patterns for Polish
syllable_pattern_pol = sy.regex_pol_ipa()

# Compiled regular expression that detects disallowed Polish characters
not_symbol_pattern_pol = regex.compile(f"[^{POLISH}]")

## II. Transcription to IPA

In [5]:
# Test 1 
sy.polish_to_ipa('kobieta')

'kɔbjɛta'

In [7]:
# Test 2
sy.polish_to_ipa('mężczyzna')

'mɛ̃ʐt͡ʂɨzna'

## III. Syllabification of a list of words

### Test the function that syllabifies a single word

In [14]:
# Test 1
s = '''- Word: {word}
- IPA transcription: {ipa}
- Syllabic cues: {syl}
'''.format(word = 'abażur', 
           ipa = sy.polish_to_ipa("abażur"), 
           syl = sy.syllabify(word = "abażur", 
                              ipa_converter = sy.polish_to_ipa, 
                              syllable_pattern = syllable_pattern_pol,
                              add_boundaries = True))
print(s)

- Word: abażur
- IPA transcription: abaʐur
- Syllabic cues: #ab_baʐ_ʐur#



In [19]:
# Test 2
s = '''- Word: {word}
- IPA transcription: {ipa}
- Syllabic cues: {syl}
'''.format(word = 'Następstwa', 
           ipa = sy.polish_to_ipa("Następstwa"), 
           syl = sy.syllabify(word = "Następstwa", 
                              ipa_converter = sy.polish_to_ipa, 
                              syllable_pattern = syllable_pattern_pol,
                              add_boundaries = True))
print(s)

- Word: Następstwa
- IPA transcription: nastɛmpstva
- Syllabic cues: #nast_stɛmpstv_mpstva#



In [20]:
# Test 3
s = '''- Word: {word}
- IPA transcription: {ipa}
- Syllabic cues: {syl}
'''.format(word = 'Szczęście', 
           ipa = sy.polish_to_ipa("Szczęście"), 
           syl = sy.syllabify(word = "Szczęście", 
                              ipa_converter = sy.polish_to_ipa, 
                              syllable_pattern = syllable_pattern_pol,
                              add_boundaries = True))
print(s)

- Word: Szczęście
- IPA transcription: ʂt͡ʂɛ̃ɕt͡ɕɛ
- Syllabic cues: #ʂt͡ʂɛ̃ɕt͡ɕ_ɕt͡ɕɛ#



### Syllabify a list of Polish words

In [9]:
# Import the list as a pandas dataframe
Words = pd.read_csv(POLISH_WORDS, names = ['Word'])
Words.head()

,Word
0,abandon
1,abażur
2,abonament
3,absolutyzm
4,absorber


In [10]:
# Apply it to the whole set
Words['IPA_transcription'] = Words['Word'].apply(lambda x: sy.polish_to_ipa(x))
Words['syllabic_cues'] = Words['Word'].apply(lambda x: sy.syllabify(x, sy.polish_to_ipa, syllable_pattern_pol))
# Display the resulting dataset
Words[:10]

,Word,IPA_transcription,syllabic_cues
0,abandon,abandɔn,#ab_band_ndɔn#
1,abażur,abaʐur,#ab_baʐ_ʐur#
2,abonament,abɔnamɛnt,#ab_bɔn_nam_mɛnt#
3,absolutyzm,absɔlutɨzm,#abs_bsɔl_lut_tɨzm#
4,absorber,absɔrbɛr,#abs_bsɔrb_rbɛr#
5,absurd,absurd,#abs_bsurd#
6,absynt,absɨnt,#abs_bsɨnt#
7,aceton,at͡sɛtɔn,#at͡s_t͡sɛt_tɔn#
8,acetylen,at͡sɛtɨlɛn,#at͡s_t͡sɛt_tɨl_lɛn#
9,adamaszek,adamaʂɛk,#ad_dam_maʂ_ʂɛk#


## IV. Syllabification of a list of sentences 

### Test the function that syllabifies a sentence

In [8]:
# Test 1 
sent1 = "Uczyłem się polskiego przez 1 miesiąc" 
s = '''- Sentence: {sent}
- IPA transcription: {ipa}
- Syllabic cues: {syl}
'''.format(sent = sent1, 
           ipa = sy.polish_to_ipa(sent1), 
           syl = sy.syllabify_line(line = sent1,
                                   ipa_converter = sy.polish_to_ipa, 
                                   syllable_pattern = syllable_pattern_pol,
                                   not_symbol_pattern = not_symbol_pattern_pol,
                                   add_boundaries = True,
                                   as_event = False))
print(s)

- Sentence: Uczyłem się polskiego przez 1 miesiąc
- IPA transcription: ut͡ʂɨwɛm ɕɛ̃ pɔlskʲɛɡɔ pʐɛz 1 mjɛɕɔnt͡s
- Syllabic cues: ['#ut͡ʂ_t͡ʂɨw_wɛm#', '#ɕɛ#', '#pɔlskʲ_lskʲɛɡ_ɡɔ#', '#pʐɛz#', '#mjɛɕ_ɕɔnt͡s#']



In [7]:
# Test 2
sent2 = "  Miło cię  spotkać!"
s = '''- Sentence: {sent}
- IPA transcription: {ipa}
- Syllabic cues: {syl}
'''.format(sent = sent2, 
           ipa = sy.polish_to_ipa(sent2), 
           syl = sy.syllabify_line(line = sent2,
                                   ipa_converter = sy.polish_to_ipa, 
                                   syllable_pattern = syllable_pattern_pol,
                                   not_symbol_pattern = not_symbol_pattern_pol,
                                   add_boundaries = True,
                                   as_event = True))
print(s)

- Sentence:   Miło cię  spotkać!
- IPA transcription:   miwɔ t͡ɕɛ̃  spɔtkat͡ɕ!
- Syllabic cues: ('#miw_wɔ#_#t͡ɕɛ#_#spɔtk_tkat͡ɕ#', 'miło_cię_spotkać')



In [6]:
# Test 3
sent3 = 'dzisiejsza słowo pochodzi z ewangelii wg św .'
s = '''- Sentence: {sent}
- IPA transcription: {ipa}
- Syllabic cues: {syl}
'''.format(sent = sent3, 
           ipa = sy.polish_to_ipa(sent3), 
           syl = sy.syllabify_line(line = sent3,
                                   ipa_converter = sy.polish_to_ipa, 
                                   syllable_pattern = syllable_pattern_pol,
                                   not_symbol_pattern = not_symbol_pattern_pol,
                                   add_boundaries = True,
                                   as_event = False))
print(s)

- Sentence: dzisiejsza słowo pochodzi z ewangelii wg św .
- IPA transcription: d͡ʑiɕɛjʂa swɔvɔ pɔxɔd͡ʑi z ɛvanɡɛlji vɡ ɕv .
- Syllabic cues: ['#d͡ʑiɕ_ɕɛjʂ_jʂa#', '#swɔv_vɔ#', '#pɔx_xɔd͡ʑ_d͡ʑi#', '#ɛv_vanɡ_nɡɛlj_lji#']



### Syllabify a set of Polish sentences

In [11]:
# Import the list as a pandas dataframe
Sentences = pd.read_csv(POLISH_SENTS)
Sentences.head()

,Sentence
0,dzisiejsza słowo pochodzi z ewangelii wg św .
1,praktycznie wszyscy z użytkowników internetu posiadają swój adres poczty e-mail .
2,szczególnie jeśli chodzi o śniadania .
3,sprzęt taki działa ale na bardzo małe odległości .
4,tunezja poza idealną pogodą posiada również liczne budowle sakralne czy medyny .


In [12]:
# Apply it to the whole set
Sentences['IPA_transcription'] = Sentences['Sentence'].apply(lambda x: sy.polish_to_ipa(x))
Sentences['syllabic_cues'] = Sentences['Sentence'].apply(lambda x: sy.syllabify_line(x, 
                                                                                     sy.polish_to_ipa, 
                                                                                     syllable_pattern_pol,
                                                                                     not_symbol_pattern_pol,
                                                                                     add_boundaries = True,
                                                                                     as_event = False))
# Display the resulting dataset
Sentences[:20]

,Sentence,IPA_transcription,syllabic_cues
0,dzisiejsza słowo pochodzi z ewangelii wg św .,d͡ʑiɕɛjʂa swɔvɔ pɔxɔd͡ʑi z ɛvanɡɛlji vɡ ɕv .,"[#d͡ʑiɕ_ɕɛjʂ_jʂa#, #swɔv_vɔ#, #pɔx_xɔd͡ʑ_d͡ʑi#, #ɛv_vanɡ_nɡɛlj_lji#]"
1,praktycznie wszyscy z użytkowników internetu posiadają swój adres poczty e-mail .,praktɨt͡ʂɲɛ vʂɨst͡sɨ z uʐɨtkɔvɲikuv intɛrnɛtu pɔɕadajɔ̃ svuj adrɛs pɔt͡ʂtɨ ɛ-majl .,"[#prakt_ktɨt͡ʂɲ_t͡ʂɲɛ#, #vʂɨst͡s_st͡sɨ#, #uʐ_ʐɨtk_tkɔvɲ_vɲik_kuv#, #int_ntɛrn_rnɛt_tu#, #pɔɕ_ɕad..."
2,szczególnie jeśli chodzi o śniadania .,ʂt͡ʂɛɡulɲɛ jɛɕli xɔd͡ʑi ɔ ɕɲadaɲa .,"[#ʂt͡ʂɛɡ_ɡulɲ_lɲɛ#, #jɛɕl_ɕli#, #xɔd͡ʑ_d͡ʑi#, #ɔ#, #ɕɲad_daɲ_ɲa#]"
3,sprzęt taki działa ale na bardzo małe odległości .,spʐɛnt takʲi d͡ʑawa alɛ na bard͡zɔ mawɛ ɔdlɛɡwɔɕt͡ɕi .,"[#spʐɛnt#, #takʲ_kʲi#, #d͡ʑaw_wa#, #al_lɛ#, #na#, #bard͡z_rd͡zɔ#, #maw_wɛ#, #ɔdl_dlɛɡw_ɡwɔɕt͡ɕ_ɕ..."
4,tunezja poza idealną pogodą posiada również liczne budowle sakralne czy medyny .,tunɛzja pɔza idɛalnɔ̃ pɔɡɔdɔ̃ pɔɕada ruvɲɛʐ lit͡ʂnɛ budɔvlɛ sakralnɛ t͡ʂɨ mɛdɨnɨ .,"[#tun_nɛzj_zja#, #pɔz_za#, #id_dɛ_aln_lnɔ̃#, #pɔɡ_ɡɔd_dɔ̃#, #pɔɕ_ɕad_da#, #ruvɲ_vɲɛʐ#, #lit͡ʂn_t..."
5,"jednakowoż nie masz zapewne jeszcze wiedzy z zakresu budownictwa , ale zaiste to nie takie skomp...","jɛdnakɔvɔʐ ɲɛ maʂ zapɛvnɛ jɛʂt͡ʂɛ vjɛd͡zɨ z zakrɛsu budɔvɲit͡stva , alɛ zajstɛ tɔ ɲɛ takʲɛ skɔmp...","[#jɛdn_dnak_kɔv_vɔʐ#, #ɲɛ#, #maʂ#, #zap_pɛvn_vnɛ#, #jɛʂt͡ʂ_ʂt͡ʂɛ#, #vjɛd͡z_d͡zɨ#, #zakr_krɛs_su#..."
6,bardzo luźny styl gwarantują marynarki sportowe bądź sweter męski recman .,bard͡zɔ luʑnɨ stɨl ɡvarantujɔ̃ marɨnarkʲi spɔrtɔvɛ bɔɲd͡ʑ svɛtɛr mɛ̃skʲi rɛt͡sman .,"[#bard͡z_rd͡zɔ#, #luʑn_ʑnɨ#, #stɨl#, #ɡvar_rant_ntuj_jɔ̃#, #mar_rɨn_narkʲ_rkʲi#, #spɔrt_rtɔv_vɛ#..."
7,w sprawach spornych obowiązuje nasz regulamin .,v spravax spɔrnɨx ɔbɔvjɔ̃zujɛ naʂ rɛɡulamin .,"[#sprav_vax#, #spɔrn_rnɨx#, #ɔb_bɔvj_vjɔ̃z_zuj_jɛ#, #naʂ#, #rɛɡ_ɡul_lam_min#]"
8,ja nie mam żadnego zaufania do takich miejsc .,ja ɲɛ mam ʐadnɛɡɔ zavfaɲa dɔ takʲix mjɛjst͡s .,"[#ja#, #ɲɛ#, #mam#, #ʐadn_dnɛɡ_ɡɔ#, #zavf_vfaɲ_ɲa#, #dɔ#, #takʲ_kʲix#, #mjɛjst͡s#]"
9,jak wszystkie ofiary mają jeden mianownik ..,jak vʂɨstkʲɛ ɔfjarɨ majɔ̃ jɛdɛn mjanɔvɲik ..,"[#jak#, #vʂɨstkʲ_stkʲɛ#, #ɔfj_fjar_rɨ#, #maj_jɔ̃#, #jɛd_dɛn#, #mjan_nɔvɲ_vɲik#]"


## V. Creating event files with syllabic cues from a Polish corpus

### Check the corpus

In [4]:
### Check the corpus
with open(CORPUS_POL) as c:
    for x in range(9):
        print(next(c))

Nadmierne napięcie grupy mięśni powoduje nieprawidłowe ustawienie w stawie . 

Mięśnie hipertoniczne mają tendencję do skracania się , co bywa jedną z przyczyn deformacji . 

Sama spastyczność nie powoduje zniekształceń , ale wywołuje ją brak równowagi między mięśniami spastycznymi a słabszymi antagonistami . 

Mięśnie przeciwdziałające mięśniom o wzmożonym napięciu pracują w niekorzystnej dla siebie sytuacji , będąc stale przez nie rozciągane . 

Nie mogą równoważyć ich siły i przez to wydają się wyraźnie słabsze . 

Z czasem rzeczywiście ulegają osłabieniu z powodu ich nieużywania . 

Koniecznie więc trzeba leczyć ten brak równowagi , zmniejszając spastyczność z jednej strony i wzmacniając mięśnie antagonistyczne . 



Mięśnie spastyczne nie są mięśniami porażonymi i mogą się kurczyć . 



### Run the syllabifier

In [5]:
# Generate the S2L event file (will also produce the corresponding L2L event file)
sy.syllabify_corpus(corpus_path = CORPUS_POL,
                    event_file_path = S2L_EVENT_POL,
                    ipa_converter = sy.polish_to_ipa,
                    syllable_pattern = syllable_pattern_pol,
                    not_symbol_pattern = not_symbol_pattern_pol,
                    add_boundaries = True,
                    numcores = 8,
                    chunksize = 125)

All processed in 0.0 minutes!tes


### Check the event file with syllabic cues

In [6]:
### Print the first 10 lines from the S2L event file 
with gzip.open(S2L_EVENT_POL, 'rt', encoding='utf-8') as e:   
    for x in range(10):
        print(next(e))

#nadmj_dmjɛrn_rnɛ#_#napj_pjɛnt͡ɕ_nt͡ɕɛ#_#ɡrup_pɨ#_#mjɛ̃ɕɲ_ɕɲi#_#pɔv_vɔd_duj_jɛ#_#ɲɛpr_prav_vidw_dwɔv_vɛ#_#ust_stavj_vjɛɲ_ɲɛ#_#stavj_vjɛ#	nadmierne_napięcie_grupy_mięśni_powoduje_nieprawidłowe_ustawienie_w_stawie

#mjɛ̃ɕɲ_ɕɲɛ#_#xip_pɛrt_rtɔɲ_ɲit͡ʂn_t͡ʂnɛ#_#maj_jɔ̃#_#tɛnd_ndɛnt͡sj_nt͡sjɛ#_#dɔ#_#skrat͡s_t͡saɲ_ɲa#_#ɕɛ#_#t͡sɔ#_#bɨv_va#_#jɛdn_dnɔ̃#_#pʐɨt͡ʂ_t͡ʂɨn#_#dɛf_fɔrm_rmat͡sj_t͡sji#	mięśnie_hipertoniczne_mają_tendencję_do_skracania_się_co_bywa_jedną_z_przyczyn_deformacji

#sam_ma#_#spast_stɨt͡ʂn_t͡ʂnɔɕt͡ɕ#_#ɲɛ#_#pɔv_vɔd_duj_jɛ#_#zɲɛkʂt_kʂtawt͡s_wt͡sɛɲ#_#al_lɛ#_#vɨv_vɔw_wuj_jɛ#_#jɔ̃#_#brak#_#ruvn_vnɔv_vaɡʲ_ɡʲi#_#mjɛnd͡z_nd͡zɨ#_#mjɛ̃ɕɲ_ɕɲam_mi#_#spast_stɨt͡ʂn_t͡ʂnɨm_mi#_#a#_#swabʂ_bʂɨm_mi#_#ant_ntaɡ_ɡɔɲ_ɲist_stam_mi#	sama_spastyczność_nie_powoduje_zniekształceń_ale_wywołuje_ją_brak_równowagi_między_mięśniami_spastycznymi_a_słabszymi_antagonistami

#mjɛ̃ɕɲ_ɕɲɛ#_#pʐɛt͡ɕ_t͡ɕivd͡ʑ_vd͡ʑaw_waj_jɔnt͡s_nt͡sɛ#_#mjɛ̃ɕɲ_ɕɲɔm#_#ɔ#_#vzmɔʐ_ʐɔn_nɨm#_#napj_pjɛnt͡ɕ_nt͡ɕu#_#prat͡s_t͡suj_jɔ̃#_#ɲɛk_kɔʐ_ʐɨstn